# Modelado - Predicción de ventas estacionales

[Link a la competencia](https://metadata.fundacionsadosky.org.ar/competition/6/)

Modelado de los datos preprocesados.

## Datos de entrada
 * */preprocessed/ventas.csv* datos de las ventas en cada POS, con información de los envíos y de los POS.
 * */ejemploRespuesta.csv*: ejemplo de respuesta para usar como ejemplo.

In [1]:
# Imports
import matplotlib.pyplot as plt
import os
import pandas as pd
import seaborn as sns

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_predict, cross_val_score, GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from IPython.display import display, Markdown as md

# Config
pd.options.display.max_columns = None

## Lectura de datos

Cargamos los archivos de entrada en *DataFrames* de *pandas*.

In [2]:
data_path = "../../data/"

In [3]:
ventas_df = pd.read_csv(os.path.join(data_path, "processed/ventas.csv"))
ejemplo_respuesta_df = pd.read_csv(os.path.join(data_path, "ejemploRespuesta.csv"), header=None, names=["id_pos", "unidades"])

In [4]:
display("Dimensiones del DataFrame: {}".format(ventas_df.shape))
display(ventas_df.head())

'Dimensiones del DataFrame: (266436, 43)'

,id_pos,fecha_anio,fecha,fecha_mes,fecha_dia,unidades,fecha_fin_de_semana,fecha_dia_trimestre,total_ventas_10_2012,total_ventas_11_2012,total_ventas_12_2012,total_ventas_10_2013,total_ventas_11_2013,total_ventas_12_2013,total_ventas_10_2014,total_ventas_11_2014,canal_ALMACEN,canal_MAXIKIOSCO,canal_PARTICULARES,canal_SUPERMERCADOS,fecha_dia_de_semana_domingo,fecha_dia_de_semana_jueves,fecha_dia_de_semana_lunes,fecha_dia_de_semana_martes,fecha_dia_de_semana_miercoles,fecha_dia_de_semana_sabado,fecha_dia_de_semana_viernes,competidores,ingreso_mediana,ingreso_promedio,densidad_poblacional,pct_secundario,pct_bicicleta,pct_omnibus,pct_subtes,pct_taxi,pct_caminata,mediana_valor_hogar,pct_ninios,pct_jovenes,pct_adultos,pct_ancianos,pct_posgrados
0,10000,2012,2012-12-03,12,3,1,False,64,0.0,0.0,12.0,0.0,0.0,8.0,0.0,0.0,1,0,0,0,0,0,1,0,0,0,0,116.0,47365.62875,63786.27125,11176.98625,28.763333,0.915,2.937917,1.39125,0.10625,4.068333,260958.6958,15.201667,29.982917,52.427083,9.827083,27.146667
1,10000,2012,2012-12-12,12,12,1,False,73,0.0,0.0,12.0,0.0,0.0,8.0,0.0,0.0,1,0,0,0,0,0,0,0,1,0,0,116.0,47365.62875,63786.27125,11176.98625,28.763333,0.915,2.937917,1.39125,0.10625,4.068333,260958.6958,15.201667,29.982917,52.427083,9.827083,27.146667
2,10000,2012,2012-12-16,12,16,1,True,77,0.0,0.0,12.0,0.0,0.0,8.0,0.0,0.0,1,0,0,0,1,0,0,0,0,0,0,116.0,47365.62875,63786.27125,11176.98625,28.763333,0.915,2.937917,1.39125,0.10625,4.068333,260958.6958,15.201667,29.982917,52.427083,9.827083,27.146667
3,10000,2012,2012-12-17,12,17,1,False,78,0.0,0.0,12.0,0.0,0.0,8.0,0.0,0.0,1,0,0,0,0,0,1,0,0,0,0,116.0,47365.62875,63786.27125,11176.98625,28.763333,0.915,2.937917,1.39125,0.10625,4.068333,260958.6958,15.201667,29.982917,52.427083,9.827083,27.146667
4,10000,2012,2012-12-18,12,18,1,False,79,0.0,0.0,12.0,0.0,0.0,8.0,0.0,0.0,1,0,0,0,0,0,0,1,0,0,0,116.0,47365.62875,63786.27125,11176.98625,28.763333,0.915,2.937917,1.39125,0.10625,4.068333,260958.6958,15.201667,29.982917,52.427083,9.827083,27.146667


## Agrupamiento por mes

In [5]:
ventas_df.columns

Index(['id_pos', 'fecha_anio', 'fecha', 'fecha_mes', 'fecha_dia', 'unidades',
       'fecha_fin_de_semana', 'fecha_dia_trimestre', 'total_ventas_10_2012',
       'total_ventas_11_2012', 'total_ventas_12_2012', 'total_ventas_10_2013',
       'total_ventas_11_2013', 'total_ventas_12_2013', 'total_ventas_10_2014',
       'total_ventas_11_2014', 'canal_ALMACEN', 'canal_MAXIKIOSCO',
       'canal_PARTICULARES', 'canal_SUPERMERCADOS',
       'fecha_dia_de_semana_domingo', 'fecha_dia_de_semana_jueves',
       'fecha_dia_de_semana_lunes', 'fecha_dia_de_semana_martes',
       'fecha_dia_de_semana_miercoles', 'fecha_dia_de_semana_sabado',
       'fecha_dia_de_semana_viernes', 'competidores', 'ingreso_mediana',
       'ingreso_promedio', 'densidad_poblacional', 'pct_secundario',
       'pct_bicicleta', 'pct_omnibus', 'pct_subtes', 'pct_taxi',
       'pct_caminata', 'mediana_valor_hogar', 'pct_ninios', 'pct_jovenes',
       'pct_adultos', 'pct_ancianos', 'pct_posgrados'],
      dtype='object')

In [6]:
# Borramos las columnas que no tienen sentido al agrupar por mes
ventas_df.drop(columns=["fecha", "fecha_dia", "fecha_fin_de_semana", "fecha_dia_trimestre", "fecha_dia_de_semana_lunes", 
                        "fecha_dia_de_semana_martes", "fecha_dia_de_semana_miercoles", "fecha_dia_de_semana_jueves", 
                        "fecha_dia_de_semana_viernes", "fecha_dia_de_semana_sabado", "fecha_dia_de_semana_domingo"], 
              inplace=True, errors="ignore")

In [7]:
ventas_df.drop(columns=['total_ventas_10_2012', 'total_ventas_11_2012', 'total_ventas_12_2012', 'total_ventas_10_2013',
                        'total_ventas_11_2013', 'total_ventas_12_2013', 'total_ventas_10_2014', 'total_ventas_11_2014'],
               inplace=True, errors="ignore")

In [8]:
ventas_df.columns

Index(['id_pos', 'fecha_anio', 'fecha_mes', 'unidades', 'canal_ALMACEN',
       'canal_MAXIKIOSCO', 'canal_PARTICULARES', 'canal_SUPERMERCADOS',
       'competidores', 'ingreso_mediana', 'ingreso_promedio',
       'densidad_poblacional', 'pct_secundario', 'pct_bicicleta',
       'pct_omnibus', 'pct_subtes', 'pct_taxi', 'pct_caminata',
       'mediana_valor_hogar', 'pct_ninios', 'pct_jovenes', 'pct_adultos',
       'pct_ancianos', 'pct_posgrados'],
      dtype='object')

In [9]:
agg_operations = {"unidades": "sum",
#                  "total_ventas_10_2012": "first",
#                  "total_ventas_11_2012": "first",
#                  "total_ventas_12_2012": "first",
#                  "total_ventas_10_2013": "first",
#                  "total_ventas_11_2013": "first",
#                  "total_ventas_12_2013": "first",
#                  "total_ventas_10_2014": "first",
#                  "total_ventas_11_2014": "first",
                 "canal_ALMACEN": "first",
                 "canal_MAXIKIOSCO": "first",
                 "canal_PARTICULARES": "first",
                 "canal_SUPERMERCADOS": "first",
                 "competidores": "first",
                 "ingreso_mediana": "first",
                 "ingreso_promedio": "first",
                 "densidad_poblacional": "first",
                 "pct_secundario": "first",
                 "pct_bicicleta": "first",
                 "pct_omnibus": "first",
                 "pct_subtes": "first",
                 "pct_taxi": "first",
                 "pct_caminata": "first",
                 "mediana_valor_hogar": "first",
                 "pct_ninios": "first",
                 "pct_jovenes": "first",
                 "pct_adultos": "first",
                 "pct_ancianos": "first",
                 "pct_posgrados": "first",
                }
    
ventas_by_month_df = ventas_df.groupby(by=["id_pos", "fecha_mes", "fecha_anio"]).agg(agg_operations)

ventas_by_month_df.reset_index(inplace=True)
display(ventas_by_month_df.head())

,id_pos,fecha_mes,fecha_anio,unidades,canal_ALMACEN,canal_MAXIKIOSCO,canal_PARTICULARES,canal_SUPERMERCADOS,competidores,ingreso_mediana,ingreso_promedio,densidad_poblacional,pct_secundario,pct_bicicleta,pct_omnibus,pct_subtes,pct_taxi,pct_caminata,mediana_valor_hogar,pct_ninios,pct_jovenes,pct_adultos,pct_ancianos,pct_posgrados
0,3142,12,2012,12,1,0,0,0,12.0,50395.31667,68263.48667,17239.673330,27.63,0.260000,6.733333,0.720000,0.210,3.793333,252050.3467,17.926667,21.003333,57.683333,10.753333,26.343333
1,3142,12,2013,7,1,0,0,0,12.0,50395.31667,68263.48667,17239.673330,27.63,0.260000,6.733333,0.720000,0.210,3.793333,252050.3467,17.926667,21.003333,57.683333,10.753333,26.343333
2,3143,11,2013,2,1,0,0,0,33.0,55256.52333,70989.74667,37567.752500,29.50,0.366667,8.410000,2.126667,0.240,3.006667,391646.2067,18.310000,18.990000,58.253333,11.313333,24.466667
3,3143,12,2013,47,1,0,0,0,33.0,55256.52333,70989.74667,37567.752500,29.50,0.366667,8.410000,2.126667,0.240,3.006667,391646.2067,18.310000,18.990000,58.253333,11.313333,24.466667
4,3144,11,2012,6,1,0,0,0,38.0,43437.32000,58236.36500,8799.223333,28.13,0.183333,1.466667,0.000000,0.025,1.518333,137570.5233,21.565000,22.411667,53.863333,9.451667,23.748333


## Random Forest Regressor
### Train y Test

In [10]:
LABEL = "unidades"
EXCLUDE_FEATURES = ["unidades", "id_pos"]
FEATURES = [col for col in ventas_by_month_df.columns.values if col not in EXCLUDE_FEATURES]

X = ventas_by_month_df.loc[:, FEATURES].values
y = ventas_by_month_df.loc[:, LABEL].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

display("Datos de entrenamiento y testeo")
display(md("X_train: {} - X_test: {}".format(X_train.shape, X_test.shape)))
display(md("y_train: {} - y_test: {}".format(y_train.shape, y_test.shape)))

'Datos de entrenamiento y testeo'

X_train: (28600, 22) - X_test: (7150, 22)

y_train: (28600,) - y_test: (7150,)

In [11]:
param_grid = {"n_estimators": (20, 50, 100),
              "max_depth": (10, 50, 100),
              "min_samples_split": [2, 5]
             }

# Búsqueda de los parámetors para RandomForest
gsc = GridSearchCV(estimator=RandomForestRegressor(),
                         param_grid=param_grid,
                         cv=3,
                         scoring="neg_mean_squared_error", # A valores más altos, mejor resultado
                         verbose=0,
                         n_jobs=-1)

grid_result = gsc.fit(X_train, y_train)
best_params = grid_result.best_params_

display("Mejores parámetros para RandomForest: {}".format(best_params))

rfr = RandomForestRegressor(n_estimators=best_params["n_estimators"], max_depth=best_params["max_depth"],
                            min_samples_split=best_params["min_samples_split"], random_state=False, verbose=False)

# K-Fold Cross Validation
scores = cross_val_score(rfr, X_train, y_train, cv=5, scoring="neg_mean_absolute_error")

print("Promedio del error absoluto medio en cross validation: {0:,.2f}".format((-scores).mean()))
print("Desviación estándar del error absoluto medio en cross validation: {0:,.2f}".format((-scores).std()))

"Mejores parámetros para RandomForest: {'max_depth': 50, 'min_samples_split': 2, 'n_estimators': 100}"

Promedio del error absoluto medio en cross validation: 11.29
Desviación estándar del error absoluto medio en cross validation: 0.15


In [12]:
rfr.fit(X_train, y_train)

atributos_importantes_df = pd.DataFrame(rfr.feature_importances_, 
                                        index = FEATURES, 
                                        columns=["importance"]).sort_values("importance", ascending=False)

display(atributos_importantes_df)

,importance
fecha_mes,2.548647e-01
pct_caminata,5.234248e-02
pct_adultos,4.981415e-02
mediana_valor_hogar,4.972310e-02
pct_ninios,4.910497e-02
pct_secundario,4.894280e-02
densidad_poblacional,4.801177e-02
pct_omnibus,4.694724e-02
pct_jovenes,4.616826e-02
pct_ancianos,4.585075e-02


In [13]:
predictions = rfr.predict(X_test)

display("Error absoluto medio: {0:,.2f}".format(mean_absolute_error(y_test, predictions)))

'Error absoluto medio: 11.00'

### Predicciones para Diciembre de 2014

In [14]:
ventas_dic_2014_df = ventas_by_month_df.drop_duplicates(subset=["id_pos"])
ventas_dic_2014_df = pd.merge(left=ejemplo_respuesta_df, right=ventas_dic_2014_df, on="id_pos", how="left")
ventas_dic_2014_df.fillna(ventas_dic_2014_df.mean(), inplace=True) # No todos los POS estan en 'pos_df'
ventas_dic_2014_df.loc[:, "fecha_mes"] = 12
ventas_dic_2014_df.loc[:, "fecha_anio"] = 2014

display(ventas_dic_2014_df.shape)
display(ventas_dic_2014_df.head())

(12897, 25)

,id_pos,unidades_x,fecha_mes,fecha_anio,unidades_y,canal_ALMACEN,canal_MAXIKIOSCO,canal_PARTICULARES,canal_SUPERMERCADOS,competidores,ingreso_mediana,ingreso_promedio,densidad_poblacional,pct_secundario,pct_bicicleta,pct_omnibus,pct_subtes,pct_taxi,pct_caminata,mediana_valor_hogar,pct_ninios,pct_jovenes,pct_adultos,pct_ancianos,pct_posgrados
0,288,2,12,2014,7.125707,0.992998,0.007002,0.0,0.0,23.903484,55052.241947,72336.831297,10209.471075,28.604281,0.575678,2.123893,0.46579,0.105386,3.379671,214389.329529,18.195530,21.072523,56.653752,10.684446,26.981075
1,953,1,12,2014,7.125707,0.992998,0.007002,0.0,0.0,23.903484,55052.241947,72336.831297,10209.471075,28.604281,0.575678,2.123893,0.46579,0.105386,3.379671,214389.329529,18.195530,21.072523,56.653752,10.684446,26.981075
2,1004,2,12,2014,7.125707,0.992998,0.007002,0.0,0.0,23.903484,55052.241947,72336.831297,10209.471075,28.604281,0.575678,2.123893,0.46579,0.105386,3.379671,214389.329529,18.195530,21.072523,56.653752,10.684446,26.981075
3,1488,4,12,2014,7.125707,0.992998,0.007002,0.0,0.0,23.903484,55052.241947,72336.831297,10209.471075,28.604281,0.575678,2.123893,0.46579,0.105386,3.379671,214389.329529,18.195530,21.072523,56.653752,10.684446,26.981075
4,3142,1,12,2014,12.000000,1.000000,0.000000,0.0,0.0,12.000000,50395.316670,68263.486670,17239.673330,27.630000,0.260000,6.733333,0.72000,0.210000,3.793333,252050.346700,17.926667,21.003333,57.683333,10.753333,26.343333


In [15]:
predictions_dic_2014 = rfr.predict(ventas_dic_2014_df.loc[:, FEATURES].values)

predictions_dic_2014_df = pd.DataFrame(index=ventas_dic_2014_df["id_pos"], data=pd.Series(predictions_dic_2014, name="unidades"))

predictions_dic_2014_df["unidades"].fillna(0, inplace=True)
predictions_dic_2014_df["unidades"] = predictions_dic_2014_df["unidades"].apply(lambda x: int(x))
print(predictions_dic_2014_df.shape)
display(predictions_dic_2014_df.head())

(12897, 1)


,unidades
id_pos,
288,46
953,30
1004,27
1488,68
3142,25


In [16]:
predictions_dic_2014_df.to_csv(os.path.join(data_path, "predictions/predictions_random_forst_regressor.csv"), 
                               index=True, 
                               header=False)